# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).


## Library Imports

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
Using PE

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': ' A/(QG E)CLNL LEBORSKADMTGASENTAS',
 'calculationPrice': 'close',
 'open': 129.41,
 'openTime': 1681388320184,
 'openSource': 'ioilfcfa',
 'close': 130.24,
 'closeTime': 1636802902935,
 'closeSource': 'oialcffi',
 'high': 131.88,
 'highTime': 1663160481917,
 'highSource': 'tua1p i5erm dlcidney ee',
 'low': 130.62,
 'lowTime': 1632585470197,
 'lowSource': '1rpd ymteecuiil5 dn aee',
 'latestPrice': 128.65,
 'latestSource': 'Close',
 'latestTime': 'December 16, 2020',
 'latestUpdate': 1624690393099,
 'latestVolume': 101486334,
 'iexRealtimePrice': 132.41,
 'iexRealtimeSize': 3,
 'iexLastUpdated': 1623332299409,
 'delayedPrice': 129.21,
 'delayedPriceTime': 1618632224305,
 'oddLotDelayedPrice': 132.6,
 'oddLotDelayedPriceTime': 1616782271723,
 'extendedPrice': 128.45,
 'extendedChange': 0,
 'extendedChangePercent': 0,
 'extendedPriceTime': 1656412039060,
 'previousClose': 133.72,
 'previousVolume': 159173228,
 'change': -

## Parsing Our API Call

In [5]:
pe_ratio = data['peRatio']
pe_ratio

39.6

## Executing A Batch API Call & Building Our DataFrame

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.13,55.6,N/A
1,AAL,17.33,-1.24,N/A
2,AAP,165.91,23.76,N/A
3,AAPL,132.65,39.42,N/A
4,ABBV,105.72,23.35,N/A
...,...,...,...,...
500,YUM,108.86,33.2,N/A
501,ZBH,154.47,-210.15,N/A
502,ZBRA,392.58,43.3,N/A
503,ZION,43.54,18.34,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [9]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [11]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,34.77,2,5752
1,NLOK,20.61,4.07,9704
2,AIV,4.74,4.53,42194
3,UNM,22.90,4.98,8733
4,BIO,592.46,5.03,337
5,KIM,15.01,6.6,13324
6,ALL,108.90,7.57,1836
7,MET,48.24,7.7,4145
8,CPB,47.62,8.17,4199
9,BIIB,258.68,8.21,773


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [13]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [14]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,217.610,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,227.910,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.85,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.350,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.430,N/A,14.85,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,56.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.250,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,137.760,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,10.280,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1782.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [17]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [19]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.863366
1      0.140594
2      0.453465
3      0.736634
4      0.457426
         ...   
500    0.665347
501    0.019802
502    0.776238
503    0.342574
504    0.833663
Name: PE Percentile, Length: 505, dtype: object
0       0.719802
1      0.0455446
2        0.39604
3       0.964356
4       0.847525
         ...    
500    0.0415842
501     0.368317
502     0.837624
503    0.0930693
504     0.942574
Name: PB Percentile, Length: 505, dtype: object
0        0.79802
1      0.0316832
2       0.158416
3       0.846535
4       0.624752
         ...    
500     0.762376
501     0.615842
502     0.689109
503      0.39604
504     0.936634
Name: PS Percentile, Length: 505, dtype: object
0       0.853465
1      0.0277228
2       0.285149
3            0.8
4       0.483168
         ...    
500     0.730693
501     0.774257
502     0.831683
503     0.231683
504     0.865347
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.819802
1      0.0574257
2       0.126733
3       0.952

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.19,N/A,54.41,0.863366,7.4000,0.719802,6.9500,0.79802,34.600726,0.853465,13.568813,0.819802,N/A
1,AAL,16.94,N/A,-1.25,0.140594,-1.8600,0.0455446,0.4386,0.0316832,-6.253920,0.0277228,1.433815,0.0574257,N/A
2,AAP,169.26,N/A,23.89,0.453465,2.9200,0.39604,1.1000,0.158416,11.386965,0.285149,2.478707,0.126733,N/A
3,AAPL,129.12,N/A,39.90,0.736634,34.6500,0.964356,8.2700,0.846535,28.067297,0.8,21.316252,0.952475,N/A
4,ABBV,107.01,N/A,23.95,0.457426,12.1300,0.847525,4.5700,0.624752,15.656635,0.483168,9.034761,0.625743,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.31,N/A,33.47,0.665347,-4.0000,0.0415842,6.1000,0.762376,23.201410,0.730693,9.920208,0.671287,N/A
501,ZBH,153.44,N/A,-206.88,0.019802,2.6400,0.368317,4.4300,0.615842,25.615702,0.774257,7.564144,0.473267,N/A
502,ZBRA,392.90,N/A,43.27,0.776238,11.0000,0.837624,4.8100,0.689109,30.056970,0.831683,11.404598,0.730693,N/A
503,ZION,43.51,N/A,17.88,0.342574,0.9792,0.0930693,2.3700,0.39604,10.500008,0.231683,2.727515,0.138614,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [20]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.19,N/A,54.41,0.863366,7.4000,0.719802,6.9500,0.79802,34.600726,0.853465,13.568813,0.819802,0.810891
1,AAL,16.94,N/A,-1.25,0.140594,-1.8600,0.0455446,0.4386,0.0316832,-6.253920,0.0277228,1.433815,0.0574257,0.0605941
2,AAP,169.26,N/A,23.89,0.453465,2.9200,0.39604,1.1000,0.158416,11.386965,0.285149,2.478707,0.126733,0.28396
3,AAPL,129.12,N/A,39.90,0.736634,34.6500,0.964356,8.2700,0.846535,28.067297,0.8,21.316252,0.952475,0.86
4,ABBV,107.01,N/A,23.95,0.457426,12.1300,0.847525,4.5700,0.624752,15.656635,0.483168,9.034761,0.625743,0.607723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.31,N/A,33.47,0.665347,-4.0000,0.0415842,6.1000,0.762376,23.201410,0.730693,9.920208,0.671287,0.574257
501,ZBH,153.44,N/A,-206.88,0.019802,2.6400,0.368317,4.4300,0.615842,25.615702,0.774257,7.564144,0.473267,0.450297
502,ZBRA,392.90,N/A,43.27,0.776238,11.0000,0.837624,4.8100,0.689109,30.056970,0.831683,11.404598,0.730693,0.773069
503,ZION,43.51,N/A,17.88,0.342574,0.9792,0.0930693,2.3700,0.39604,10.500008,0.231683,2.727515,0.138614,0.240396


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [21]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [22]:
portfolio_input()

Enter the value of your portfolio:200000000


In [23]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/Users/caseybrugna/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.18,172562,4.84,0.156436,0.4323,0.0475248,0.3890,0.0277228,2.638174,0.0336634,0.397242,0.0039604,0.0538614
1,AIG,39.57,101086,-6.95,0.10495,0.5325,0.0554455,0.7341,0.0653465,4.392891,0.0415842,0.734063,0.0138614,0.0562376
2,AAL,16.94,236127,-1.25,0.140594,-1.8600,0.0455446,0.4386,0.0316832,-6.253920,0.0277228,1.433815,0.0574257,0.0605941
3,MET,47.53,84157,7.87,0.164356,0.5843,0.0574257,0.6482,0.0574257,4.341707,0.039604,0.621150,0.0118812,0.0661386
4,HFC,26.57,150545,-26.50,0.0673267,0.8136,0.0772277,0.3076,0.0138614,6.345672,0.0693069,2.208429,0.116832,0.0689109
5,F,9.28,431034,-17.58,0.0792079,1.2000,0.132673,0.2763,0.0118812,4.933697,0.0455446,2.099702,0.108911,0.0756436
6,HIG,46.38,86244,9.77,0.182178,0.9516,0.0891089,0.8122,0.0821782,4.007343,0.0376238,0.795814,0.0217822,0.0825743
7,ALL,110.12,36324,7.63,0.162376,1.3200,0.161386,0.7370,0.0673267,2.480937,0.0316832,0.740495,0.0158416,0.0877228
8,KSS,40.91,97775,-58.00,0.049505,1.2800,0.152475,0.3577,0.0237624,9.373370,0.168317,1.363958,0.0534653,0.089505
9,BA,232.35,17215,-29.57,0.0653465,-11.3100,0.0277228,2.1500,0.360396,-30.797415,0.0138614,-222.360642,0.0019802,0.0938614


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [24]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [27]:
writer.save()